In [1]:
import pandas as pd
import numpy as np

# ==========================================
# 1. LOAD THE DATA
# ==========================================
# In Colab, you would upload the files. For this demo, I will create the dataframes directly
# to match your exact data so you can run it instantly without uploading files.

# --- DATASET A: XERO EXPORT (The data you provided) ---
xero_data = {
    'Date': ['02 Apr 2025', '03 May 2025', '02 Jun 2025', '02 Jun 2025', '03 Jul 2025',
             '02 Aug 2025', '02 Aug 2025', '02 Sep 2025', '11 Sep 2025', '11 Sep 2025',
             '11 Sep 2025', '11 Sep 2025', '12 Sep 2025', '13 Sep 2025'],
    'Invoice_Number': ['INV-0005', 'INV-0006', 'INV-0007', 'INV-0007', 'INV-0008',
                       'INV-0009', 'INV-0009', 'INV-0010', 'INV-0015', 'INV-0013',
                       'INV-0012', 'INV-0014', 'INV-0016', 'CN-0025'],
    'Reference': ['RPT200-1', 'RPT200-1', 'RPT200-1', 'RPT200-1', 'RPT200-1',
                  'RPT200-1', 'RPT200-1', 'RPT200-1', 'Monthly Support', 'Monthly Support',
                  'Monthly Support', 'Monthly Support', 'Monthly Support', 'Monthly Support'],
    'Client': ['Ridgeway', 'Ridgeway', 'Ridgeway', 'Ridgeway', 'Ridgeway',
               'Ridgeway', 'Ridgeway', 'Ridgeway', 'Rex Media', 'Young Bros',
               'Hamilton Smith', 'Port & Philip', 'Hamilton Smith', 'Hamilton Smith'],
    'Net_Amount': [416.67, 416.67, 833.33, 416.67, 416.67,
                   583.33, 416.67, 416.67, 451.04, 451.04,
                   451.04, 451.04, 451.04, -451.04] # Note the credit note is negative
}
df_xero = pd.DataFrame(xero_data)

# --- DATASET B: PROJECT LOG (The "Step 2" data I created) ---
project_data = {
    'Project_Ref': ['RPT200-1', 'RPT200-1', 'RPT200-1', 'RPT200-1', 'RPT200-1', 'RPT200-1',
                    'Monthly Support', 'Monthly Support', 'Monthly Support', 'Monthly Support', 'P/O 9999'],
    'Client_Name': ['Ridgeway', 'Ridgeway', 'Ridgeway', 'Ridgeway', 'Ridgeway', 'Ridgeway',
                    'Rex Media', 'Young Bros', 'Hamilton Smith', 'Port & Philip', 'Future Growth Ltd'],
    'Expected_Fee': [416.67, 416.67, 1250.00, 416.67, 1000.00, 416.67,
                     451.04, 451.04, 500.00, 451.04, 2000.00],
    'Month_Tag': ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Sep', 'Sep', 'Sep', 'Sep', 'Sep']
}
df_project = pd.DataFrame(project_data)

# ==========================================
# 2. CLEANING & AGGREGATION (The "Messy Data" Solution)
# ==========================================
print("--- 🧹 STEP 1: CLEANING DATA ---")

# Problem: Xero has multiple lines for one invoice (e.g., Ridgeway in June has 2 lines).
# Solution: Group Xero data by 'Reference' and 'Date' to match the Project Log.

# Grouping Xero Data
xero_grouped = df_xero.groupby(['Reference', 'Client'])['Net_Amount'].sum().reset_index()
print(f"Compressed Xero Lines from {len(df_xero)} down to {len(xero_grouped)} unique billable items.")

# Grouping Project Data (just in case)
project_grouped = df_project.groupby(['Project_Ref', 'Client_Name'])['Expected_Fee'].sum().reset_index()

# ==========================================
# 3. THE MATCHING ENGINE
# ==========================================
print("\n--- 🔍 STEP 2: RECONCILING ---")

# We match 'Reference' from Xero to 'Project_Ref' from Project Log
merged = pd.merge(project_grouped, xero_grouped,
                  left_on='Project_Ref',
                  right_on='Reference',
                  how='left')

# Calculate Variance
merged['Variance'] = merged['Net_Amount'] - merged['Expected_Fee']

# ==========================================
# 4. EXCEPTION REPORT
# ==========================================
def flag_status(row):
    if pd.isna(row['Net_Amount']):
        return "🚨 MISSING: Work done but NOT Invoiced"
    elif abs(row['Variance']) > 1.0: # Allow £1 rounding difference
        return f"⚠️ VARIANCE: Difference of £{row['Variance']:.2f}"
    else:
        return "✅ MATCHED"

merged['Status'] = merged.apply(flag_status, axis=1)

print("\n--- 📊 FINAL REVENUE ASSURANCE REPORT ---")
display_cols = ['Client_Name', 'Project_Ref', 'Expected_Fee', 'Net_Amount', 'Status']
print(merged[display_cols].to_markdown(index=False))

--- 🧹 STEP 1: CLEANING DATA ---
Compressed Xero Lines from 14 down to 5 unique billable items.

--- 🔍 STEP 2: RECONCILING ---

--- 📊 FINAL REVENUE ASSURANCE REPORT ---
| Client_Name       | Project_Ref     |   Expected_Fee |   Net_Amount | Status                                 |
|:------------------|:----------------|---------------:|-------------:|:---------------------------------------|
| Hamilton Smith    | Monthly Support |         500    |       451.04 | ⚠️ VARIANCE: Difference of £-48.96     |
| Hamilton Smith    | Monthly Support |         500    |       451.04 | ⚠️ VARIANCE: Difference of £-48.96     |
| Hamilton Smith    | Monthly Support |         500    |       451.04 | ⚠️ VARIANCE: Difference of £-48.96     |
| Hamilton Smith    | Monthly Support |         500    |       451.04 | ⚠️ VARIANCE: Difference of £-48.96     |
| Port & Philip     | Monthly Support |         451.04 |       451.04 | ✅ MATCHED                             |
| Port & Philip     | Monthly Support |   